In [76]:
import pandas as pd
import numpy as np

In [42]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
import asyncpraw

In [55]:
reddit = await asyncpraw.Reddit(
    client_id=os.environ["REDDIT_CLIENT_ID"],
    client_secret=os.environ["REDDIT_CLIENT_SECRET"],
    # password=os.environ.get("REDDIT_PASSWORD"),
    user_agent=os.environ["USER_AGENT"]
    #, username=os.envirton.get("REDDIT_USERNAME")
).__aenter__()

# reddit.read_only = True|False (if using OAuth, default=False if OAuth is used)

### Ratelimits

In [45]:
print(reddit.auth.limits['used']) # if connected to OAuth

None


In [46]:
print(reddit.auth.limits['remaining']) # if connected to OAuth

None


In [47]:
print(reddit.auth.limits['reset_timestamp']) # if connected to OAuth

None


### Data retrieval

In [56]:
wsb = await reddit.subreddit("wallstreetbets")

In [86]:
submission_df = []

In [87]:
submissions = wsb.new(limit=1) # must recreate for each async iteration
async for submission in submissions:
    print(submission.selftext)
    await submission._fetch()
    submission_comments = []
    async for comment in submission.comments:
        submission_comments.append(comment.body)
    submission_df.append([submission.title, submission.selftext, submission_comments])

hopefully i can come out of this unscathed...idk what i was thinking.


In [88]:
pd.DataFrame(submission_df)

,0,1,2
0,wont be doing this BS again,hopefully i can come out of this unscathed...i...,[\n**User Report**| | | |\n:--|:--|:--|:--\n**...


In [89]:
await reddit.close()